$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$

$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

$$\newcommand{\ketbra}[2]{\ket{#1}\bra{#2}}$$



# Import de Modules

In [ ]:
from qutip import *

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from scipy.fft import fft, fftfreq


plt.switch_backend("module://ipykernel.pylab.backend_inline")
plt.rcParams.update({'font.size': 32})

# 1. Système à deux niveaux avec drive

### Définition des vecteurs de base

In [ ]:
etat_g = basis(2,0)  # |g>
etat_e = basis(2,1)  # |e>

gg = etat_g*etat_g.dag()  # |g><g|
ee = etat_e*etat_e.dag()  # |e><e|
ge = etat_g*etat_e.dag()  # |e><e|
eg = etat_e*etat_g.dag()  # |e><g|

### Définition du Hamiltonien et probabilité de transition ($P_{g\to e}(t)$)




Rappel théorique: 

$$\boxed{P_{g \rightarrow e}(t)=\frac{|\Omega|^2}{|\Omega|^2+\Delta^2} \sin ^2\left(t / 2 \sqrt{\Delta^2+|\Omega|^2}\right)}$$


In [ ]:
Delta = 1
drive = 1
psi0 = etat_g

# -------------------

H0 = ?

Hint = ?

# -------------------

H = H0 + Hint

T = 2*np.pi/abs(drive) # Une "periode" d'oscillation (pour drive >> Delta)

tmax = T
t = np.linspace(0,tmax,101)
            
result = mesolve(H, psi0, t) # permet d'obtenir la matrice densité/vecteur d'état pour chaque points en temps

P_ge = expect(ee ,result.states) # P<phi(t)|e><e|phi(t)> 

# print(result.states)

In [ ]:
plt.plot(t, P_ge, label=r'$P_{g \rightarrow e} (t)$')
plt.legend()
plt.xticks([0,tmax/2, tmax], ["$0$", r"$\pi"+f"/{int(T/(tmax))}$", r"$\pi"+f"/{int(T/2/tmax)}$"])
plt.title(f"$\Delta={Delta}, \Omega={drive}$")
plt.show()

### Visualisation sur la sphère de Bloch

In [ ]:
b = qutip.Bloch()

# Système à 3 niveau (Qutrit) (ou Qubit à faible anharmonicité)

# Type d'Impulsions 

In [ ]:
# échelle de temps sur laquelle calculer l'évolution

max_time = 5 # evolution temporelle entre 0 et max_time 
t = np.linspace(0,max_time,1001)

# option pour s'assurer que mesolve n'ignore pas le changement de cham
mesolve_max_step = 1e-1

### Impulsion rectengulaire

In [ ]:
# définition de l'enveloppe du champ de pompe:
# cette enveloppe varie entre 0 et 1
pulse_width = 1  # impulsion pi/2 en seconde

def rect_enveloppe(t):
    return np.heaviside(t-max_time/2+pulse_width/2, 1)*np.heaviside(max_time/2-t+pulse_width/2, 1)

plt.plot(t,rect_enveloppe(t))
plt.xlabel("$t$ (s)")
plt.show()

### (Re)défintion du Hamiltonien

In [ ]:
# difference en fréquence entre les transitions |g> -> |e> et |e> -> |f>
anharmonicite = -0.05

# décalage spectral entre la pompe et la transition |g> -> |e>
delta = 0.0

drive = 1/(4*pulse_width) * 2 * np.pi

H0 = delta * ee3 + (delta+anharmonicite) * ff3 

Hint = (drive/2 * eg3 + np.conjugate(drive)/2 * ge3) + (drive/2 * fe3 + np.conjugate(drive)/2 * ef3)

def H(t, args):
    enveloppe = args[0]
    """ Fonction permettant de calculer le Hamiltonien à chaque temps t. """
    return H0 + enveloppe(t)*Hint


In [ ]:
result = mesolve(hamiltonian_t, etat3_g, time, [],[], args=[rect_enveloppe], options=Options(max_step=mesolve_max_step))


plt.plot(t, except(ee3, result.states))

### Fidélité

In [ ]:
fidelite_rect = [fidelity(1/np.sqrt(2)*(etat3_g-1j*etat3_e), state) for state in result.states]
plt.plot(time,fidelite_rect)
plt.ylabel('Fidélité')
plt.xlabel('Temps (ns)')
plt.show()
plt.semilogy(time*1e9,1-np.array(fidelite_rect))
plt.ylabel('Infidélité')
plt.xlabel('Temps (ns)')
plt.show()

### Impulsion gaussienne

In [ ]:
alpha = 1.25
gaus_width = pulse_width/(2*alpha)
def gaus_enveloppe(t):
    return np.exp(-(t-max_time/2)**2/(2*gaus_width**2))

plt.plot(t, gaus_enveloppe(t))
plt.xlabel("$t$ (s)")
plt.show()

result_gauss = mesolve(hamiltonian_t, etat3_g, time, [],[], args=[gaus_enveloppe], options=Options(max_step=mesolve_max_step))
plt.plot(t, except(ee3, result_gauss.states))


In [ ]:
fidelite_gauss = [fidelity(1/np.sqrt(2)*(etat3_g-1j*etat3_e), state) for state in result_gauss.states]
plt.plot(time,fidelite_gauss)
plt.ylabel('Fidélité')
plt.xlabel('Temps (ns)')
plt.show()
plt.semilogy(time,1-np.array(fidelite_gauss))
plt.ylabel('Infidélité')
plt.xlabel('Temps (ns)')
plt.show()